# Optimizing training and inference

In this notebook, we will discuss different ways to reduce memory and compute usage during training and inference.

## Prepare training script

When training large models, it is usually a best practice not to use Jupyter notebooks, but run a **separate script** for training which could have command-line flags for various hyperparameters and training modes. This is especially useful when you need to run multiple experiments simultaneously (e.g. on a cluster with task scheduler). Another advantage of this is that after training, the process will finish and free the resources for other users of a shared GPU.

**Task**. In this part, you will need to put all your code to train a model on CIFAR-10 that you wrote for the previous task in `train.py`. Your code has to run for at least one epoch (a single run over the whole dataset).

You can then run your script from inside of this notebook like this:

In [ ]:
!python3 train.py

## Profiling time

For the next tasks, you need to add measurements to your training loop. You can use [`perf_counter`](https://docs.python.org/3/library/time.html#time.perf_counter) for that:

In [ ]:
import time
import numpy as np
import torch

In [ ]:
x = np.random.randn(1000, 1000)
y = np.random.randn(1000, 1000)

start_counter = time.perf_counter()
z = x @ y
elapsed_time = time.perf_counter() - start_counter
print(f"Matrix multiplication took {elapsed_time:.3f} seconds")

**Task**. You need to add the following measurements to your training script:
* How much time a forward-backward pass takes for a single batch;
* How much time an epoch takes.

## Profiling memory usage

**Task**. You need to measure the memory consumptions

This section depends on whether you train on CPU or GPU.

### If you train on CPU
You can use GNU time to measure peak RAM usage of a script:

In [ ]:
!/usr/bin/time -lp python train.py

**Maximum resident set size**  will show you the peak RAM usage in bytes after the script finishes.

### If you train on GPU

Use [`torch.cuda.max_memory_allocated()`](https://pytorch.org/docs/stable/cuda.html#torch.cuda.max_memory_allocated) at the end of your script to show the maximum amount of memory in bytes used by all tensors.

In [ ]:
x = torch.randn(1000, 1000, 1000, device='cuda:0')
print(f"Peak memory usage by Pytorch tensors: {(torch.cuda.max_memory_allocated() / 1024 / 1024):.2f} Mb")

## Gradient based techniques

Modern architectures can potentially consume lots and lots of memory even for minibatch of several objects. To handle such cases here we will discuss two simple techniques.

### Gradient Checkpointing

Checkpointing works by trading compute for memory. Rather than storing all intermediate activations of the entire computation graph for computing backward, the checkpointed part does not save intermediate activations, and instead recomputes them in backward pass. It can be applied on any part of a model.

See [blogpost](https://medium.com/tensorflow/fitting-larger-networks-into-memory-583e3c758ff9) for kind introduction and different strategies or [article](https://arxiv.org/pdf/1604.06174.pdf) for not kind introduction.

**Task**. Use [built-in checkpointing](https://pytorch.org/docs/stable/checkpoint.html), measure the difference in memory/compute 


### Accumulating gradient for large batches
We can increase the effective batch size by simply accumulating gradients over multiple forward passes. Note that `loss.backward()` simply adds the computed gradient to `tensor.grad`, so we can call this method multiple times before actually taking an optimizer step. However, this approach might be a little tricky to combine with batch normalization. Do you see why?

In [ ]:
effective_batch_size = 1024
loader_batch_size = 32
batches_per_update = effective_batch_size / loader_batch_size # Updating weights after 8 forward passes

dataloader = DataLoader(dataset, batch_size=loader_batch_size)

optimizer.zero_grad()

for batch_i, (batch_X, batch_y) in enumerate(dataloader):
    l = loss(model(batch_X), batch_y)
    l.backward() # Adds gradients
  
    if (batch_i + 1) % batches_per_update == 0:
        optimizer.step()
        optimizer.zero_grad()

**Task**. Explore the trade-off between computation time and memory usage while maintaining the same effective batch size. By effective batch size we mean the number of objects over which the loss is computed before taking a gradient step.

## Accuracy vs compute trade-off

### Tensor type size

One of the hyperparameter affecting memory consumption is the precision (e.g. floating point number). The most popular choices are 32 bit and 64 bit precision. 

**Task**. Let's turn to your code and analyze how the change of the precision will affect the perfomance and memory. Here two cases:
* if you have FloatTensor -> try DoubleTensor. Does the increased precision affect the perfomance?
* if you have DoubleTensor -> try FloatTensor. Did you lose perfomance? (in fact, you can even gain!)

**Note**.  Training with 16 bit precision is called mixed precision training. 16 bit arithmetics is not implemented by default. Several hacks (https://arxiv.org/pdf/1710.03740.pdf) allow it to work giving you memory gain for free.

### Quantization

We can actually move further and use even lower precision like 8-bit integers:

* https://heartbeat.fritz.ai/8-bit-quantization-and-tensorflow-lite-speeding-up-mobile-inference-with-low-precision-a882dfcafbbd
* https://nervanasystems.github.io/distiller/quantization/
* https://arxiv.org/abs/1712.05877

### Knowledge distillation
Suppose that we have a large network (*teacher network*) or an ensemble of networks which has a good accuracy. We can like train a much smaller network (*student network*) using the outputs of teacher networks. It turns out that the perfomance could be even better! This approach doesn't help with training speed, but can be quite beneficial when we'd like to reduce the model size for low-memory devices.

* https://www.ttic.edu/dl/dark14.pdf
* [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531)
* https://medium.com/neural-machines/knowledge-distillation-dc241d7c2322

**Task**. Distill your (teacher) network with smaller one (student), compare it perfomance with the teacher network and with the same (student) trained directly from data

**Note**. You can even use a completely different ([article](https://arxiv.org/abs/1711.10433)) architecture in a student model, e.g. approximate an autoregressive model (WaveNet) by a non-autoregressive one.

This approach doesn't help with training speed, but can be quite beneficial when we'd like to reduce the model size for low-memory devices.